# Autoencoders

En las próximas prácticas vamos a crear arquitecturas propias del aprendizaje no supervisado. Como ya sabrás, este tipo de problemas se caracterizan por la falta de etiquetas en los datos, lo que significa que no contamos con una respuesta correcta para cada entrada.

En su lugar, buscamos descubrir patrones, estructuras y relaciones dentro de los datos. Una de las redes neuronales profundas más utilizadas para este tipo de problemas son los **Autoencoders**.

Un Autoencoder es un tipo de red neuronal utilizada para aprender representaciones eficientes (codificaciones) de los datos, típicamente para la reducción de dimensionalidad.

Durante su entrenamiento tienen como objetivo reconstruir su entrada en la salida, pasando la información a través de una red de capas más pequeñas.

### Estructura de un Autoencoder

![image](https://i.imgur.com/0gIK8Gd.png)

Estos constan de dos partes principales:

1. **Codificador (Encoder)**: Comprime la entrada a un espacio *latente* de menor dimensión que el espacio original.
2. **Decodificador (Decoder)**: Reconstruye los datos originales a partir de la representación compacta creada por el codificador.

En esta práctica entrenaremos un Autoencoder convolucional capaz de aprender una codificación en un espacio de 2D ($\mathbb{R}^{2}$) para cada una de las imágenes de un conjunto.


## Conjunto de datos

En este caso hemos optado por utilizar el conjunto [MNIST](https://es.wikipedia.org/wiki/Base_de_datos_MNIST). Este conjunto posee imágenes en escala de grises con números del 0 al 9 escritos a mano por personas. Cada una de estas imágenes tiene una resolución de $28 \times 28$ píxels. Al ser en escala de grises, cada imagen será un tensor de $1 \times 28 \times 28$.

Este conjunto de datos está pensado para resolver un problema de *Aprendizaje supervisado de multiclasificación*, pero en este caso descartaremos las etiquetas y utilizaremos solamente las imágenes.

### Descargar conjunto

Utilizaremos el módulo `datasets` de `keras` para obtener el conjunto.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow.keras.datasets import mnist

# Fijar la semilla para obtener reproducibilidad
seed = 42
keras.utils.set_random_seed(seed)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Como puedes ver a continuación, este conjunto está formado por 60000 imágenes de train y 10000 de test.

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Las imágenes vienen sin normalizar, por tanto lo primero será pasarlas del rango $[0,255]$ al rango $[0,1]$ y luego crearemos los datasets.

In [ ]:
# Normalizar los datos al rango [0, 1]
x_train, x_test = x_train /255, x_test / 255

# Crear los datasets de tensorflow
train_data = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, x_test))

## Autoencoder Convolucional

Los autoencoders no poseen una arquitectura definida; esta depende del problema y del tipo de datos con los que trabajemos. Lo único que definen es la necesidad de crear una parte *encoder* y otra *decoder*.

Al trabajar con imágenes, nuestro encoder estará formado por capas convolucionales que procesarán la imagen y la proyectarán en un espacio de, en este caso, $\mathbb{R}^{2}$. 

El decoder hará el proceso inverso: deberá generar, dado un vector 2D, una salida de $1 \times 28 \times 28$. Para llevar a cabo este proceso serán necesarias capas denominadas *deconvoluciones* o *convoluciones transpuestas*.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_autoencoder(k=2, learning_rate=1e-4):

    # Definimos el encoder
    input_img = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(8, (3, 3), strides=2, padding='same', activation='tanh')(input_img)  # (batch, 14, 14, 8)
    x = layers.Conv2D(16, (3, 3), strides=2, padding='same', activation='tanh')(x)  # (batch, 7, 7, 16)
    x = layers.Conv2D(32, (3, 3), strides=2, padding='same', activation='tanh')(x)  # (batch, 4, 4, 32)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='tanh')(x)  # Proyección a vector de 64D
    encoded = layers.Dense(k)(x)  # Proyección a vector de K

    encoder = models.Model(input_img, encoded, name='encoder')

    # Definimos el decoder
    encoded_input = layers.Input(shape=(k,))
    x = layers.Dense(64, activation='tanh')(encoded_input)
    x = layers.Dense(16 * 7 * 7, activation='tanh')(x)
    x = layers.Reshape((7, 7, 16))(x)
    x = layers.Conv2DTranspose(8, (3, 3), strides=2, padding='same', activation='tanh')(x)  # (batch, 16, 16, 8)
    decoded = layers.Conv2DTranspose(1, (3, 3), strides=2, padding='same', activation='sigmoid', output_padding=1)(x)  # (batch, 28, 28, 1)

    decoder = models.Model(encoded_input, decoded, name='decoder')
    
    # Conectar el encoder y el decoder para crear el autoencoder completo
    autoencoder_input = layers.Input(shape=(28, 28, 1))
    encoded_output = encoder(autoencoder_input)
    decoded_output = decoder(encoded_output)

    autoencoder = models.Model(autoencoder_input, decoded_output, name='autoencoder')
    autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='mse')
    
    return encoder, decoder, autoencoder

In [ ]:
# Crear autoencoder que proyecta en k=2
_, _, autoencoder = create_autoencoder()

# Resumen del modelo autoencoder
autoencoder.summary()

### Callback
A continuación vamos a crear un `callback` para visualizar, tras cada epoch, las imagenes reales y las predichas por el autoencoder.

In [ ]:
import matplotlib.pyplot as plt 

class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data
    
    def on_epoch_end(self, epoch, logs=None):
        # Borrar la consola
        clear_output(wait=True)
        # Seleccionar un lote del conjunto de prueba
        for test_batch in self.test_data.take(1):
            test_images, _ = test_batch

            # Generar imágenes utilizando el autoencoder
            generated_images = self.model(test_images)
            
            # Mostrar las imágenes originales y generadas
            fig, axs = plt.subplots(2, 10, figsize=(10, 2))
            for i in range(10):
                # Mostrar la imagen original
                axs[0, i].imshow(test_images[i], cmap='gray')
                axs[0, i].axis('off')
                
                # Mostrar la imagen generada
                axs[1, i].imshow(generated_images[i], cmap='gray')
                axs[1, i].axis('off')
            
            plt.suptitle(f'Epoch {epoch + 1}')
            plt.show()

### Entrenar autoencoder con $k=2$

In [ ]:
# Configuración de parámetros
batch_size = 64
learning_rate = 0.0005
num_epochs = 25

# Crear el autoencoder
encoder, decoder, autoencoder = create_autoencoder(k=2, learning_rate=learning_rate)

# Definir los batches
train_dataset = train_data.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
test_dataset = test_data.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

# Inicializamos el callcback previo
callback = DisplayCallback(test_dataset)

# Entrenamiento del modelo
history = autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=num_epochs, verbose=1, callbacks=[callback])

Al reducir a un espacio de 2 dimensiones, podemos visualizar cada imagen como un punto en un gráfico de dispersión coloreando cada uno según el dígito que representa.

In [ ]:
import matplotlib.pyplot as plt 

image_embs = encoder.predict(test_dataset)

# Crear el scatterplot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(image_embs[:, 0], image_embs[:, 1], c=y_test, alpha=0.7)
plt.title('Scatterplot de las características codificadas')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend()
plt.show()

### Entrenar autoencoder con $k=8$
Como se ha visto, reducir a un espacio de $k=2$ permite reconstruir las imágenes originales, pero a costa de perder cierta calidad. Probaremos a continuación la reducción a espacios de mayor dimensión.

In [ ]:
# Conectar el encoder y el decoder para crear el autoencoder completo
encoder, decoder, autoencoder = create_autoencoder(k=8, learning_rate=learning_rate)

# Entrenamiento del modelo
history = autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=num_epochs, verbose=1, callbacks=[callback])

### Entrenar autoencoder con $k=16$

In [ ]:
# Conectar el encoder y el decoder para crear el autoencoder completo
encoder, decoder, autoencoder = create_autoencoder(k=16, learning_rate=learning_rate)

# Entrenamiento del modelo
history = autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=num_epochs, verbose=1, callbacks=[callback])

## Interpolación
Para finalizar vamos a utilizar el decoder para generar nuevas imágenes por interpolación. Básicamente tendremos que obtener el vector de dos imágenes reales utilizando el encoder y luego generar $n$ vectores intermedios y ver que imágenes resultan utilizando el decoder.

In [ ]:
start = x_test[-1] # Una imagen de un 6
end = x_test[3] # Una imagen de un 0

# Obtener su proyección en el espacio k
start_emb = encoder(tf.expand_dims(start, 0))
end_emb = encoder(tf.expand_dims(end, 0))

# Número de puntos total incluidos extremos
x = 10
# Generar puntos equidistantes entre start_emb y end_emb
interpolated_embs = tf.squeeze(tf.linspace(start_emb, end_emb, num=x))
# Obtener imágenes resultantes
images = decoder(interpolated_embs)

plt.figure(figsize=(x*1.5, 4))

for idx, im in enumerate(images): 
    # Imagen original
    plt.subplot(1, len(images), idx+1)
    plt.imshow(im, cmap='gray')
    plt.axis('off')